# 19. 基于Mindspore构造全连接层

本实验主要介绍使用MindSpore构造全连接层。并使用人为提供数据进行计算。

# 1 实验目的

- 了解全连接层的原理
- 掌握如何使用MindSpore构造全连接层

# 2 全连接层原理介绍

全连接层指的是层中的每个节点都会连接它下一层的所有节点，它是模仿人脑神经结构来构建的。用来把前边提取到的特征综合起来。由于其全相连的特性，一般全连接层的参数也是最多的。全连接层则起到将学到的“分布式特征表示”映射到样本标记空间的作用。

公式如下：
$$output=activation(X*kernel+bias)$$
其中 X 是输入Tensor， activation 是激活函数， kernel 是一个权重矩阵，其数据类型与 X 相同， bias 是一个偏置向量，其数据类型与 X 相同（仅当has_bias为True时）。

具有两个隐藏层的全连接层结构如下：

![jupyter](./Figures/Fig001.png)

# 3 实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|


# 4 数据处理

## 4.1 数据准备
自生成x，x为两行三列的矩阵
$$x = [[180, 234, 154], [244, 48, 247]]$$




## 4.2 数据加载

使用numpy和Tensor生成x。

In [16]:
from mindspore import Tensor                # 张量
import numpy as np                          # 科学计算库
from mindspore import dtype as mstype       # 数据类型模块 
# 构建一个2*3的矩阵并转换为Tensor
x = Tensor(np.array([[180, 234, 154], [244, 48, 247]]), mstype.float32)

# 5 模型构建

在使用前，导入需要的Python库，ops模块定义了一些运算，我们需要这些API，Cell模块是网络基类模块，我们将要构建的Dense便继承与它，其他模块不在一一说明，详细的MindSpore的模块说明，可以在MindSpore API页面中搜索查询。



In [42]:
import mindspore.nn as nn                              # 神经网络模块                            
from mindspore import Parameter                        # 封装权重模块，初始化后的Parameter是Tensor的子类
from mindspore import ops                              # 常见算子操作
from mindspore.common.initializer import initializer   # 初始化神经元参数
from mindspore.nn import get_activation                #获取激活函数模块

具体实现 Dense API

通过继承Cell类，并重写__init__方法和construct方法，来构造全连接层。

In [43]:
class Dense(nn.Cell):
    # 初始化全连接层
    def __init__(self,
                 in_channels,           #输入Tensor的空间维度
                 out_channels,          #输出Tensor的空间维度
                 weight_init='normal',  #权重参数的初始化方法，采用normal
                 bias_init='zeros',     #偏置参数的初始化方法，采用zeros
                 has_bias=True,         #是否使用偏置向量 bias，默认为True
                 activation=None        #应用于全连接层输出的激活函数，采用None
                 ):
        #调用父类初始化函数完成初始化        
        super(Dense, self).__init__()

        self.reshape = ops.Reshape()  #使用ops模块的Reshape，Reshape基于给定的shape，对输入Tensor进行重新排列
        self.shape_op = ops.Shape()   #使用ops模块的Shape，Shape返回输入Tensor的shape
        self.matmul = ops.MatMul(transpose_b=True)    #使用ops模块的MatMul
        self.weight = Parameter(initializer(weight_init, [out_channels, in_channels]), name="weight")   #初始化权重
        self.bias = None    #bias初始为空
        self.has_bias = has_bias
        if self.has_bias:
            self.bias = Parameter(initializer(bias_init, [out_channels]), name="bias")
            self.bias_add =ops.BiasAdd()     #使用ops模块的BiasAdd，将来用于加上偏置

        #定义激活函数 如果activation是字符类型则获取相应激活函数
        #否则意味activation对象是一个函数，然后将其赋值给self.activation
        self.activation = get_activation(activation) if isinstance(activation, str) else activation     
        self.activation_flag = self.activation is not None    #判断是否设置激活函数，从而设置激活函数的flag

    def construct(self, x):
        x_shape = self.shape_op(x)  #获得x的shape

        if len(x_shape) != 2:       #若x的维度不是2，对x进行调整，x的一个维度是数据的个数
            x = self.reshape(x, (-1, x_shape[-1]))
        x = self.matmul(x, self.weight)    #x与权重
        if self.has_bias:           #若使用偏置，对偏置与x进行相加
            x = self.bias_add(x, self.bias)
        if self.activation_flag:    #若使用激活函数，传入x进行计算
            x = self.activation(x)
        if len(x_shape) != 2:       #调整输出格式
            out_shape = x_shape[:-1] + (-1,)
            x = self.reshape(x, out_shape)
        return x

# 6 模型测试

构建一个简单的全连接层（3输入4输出），将x传入进行计算。

In [44]:
# 构建输入为3个节点，输出为4个节点的全连接层
net = Dense(3, 4)
# 输出为2*4矩阵
output = net(x)
print(output)
print(output.shape)

[[1.833653   2.4907696  6.631096   0.41210198]
 [2.0204396  1.0070385  9.949867   4.0580835 ]]
(2, 4)
